In [251]:
import pandas as pd
from matplotlib import pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [296]:
enchanting = pd.read_csv('enchanting.csv')

In [297]:
enchanting.head()

,luck,item,item_level,enchanting_level,result
0,NAN,accessory,0,0,fail
1,NAN,accessory,0,1,fail
2,NAN,accessory,0,2,fail
3,NAN,accessory,0,3,fail
4,NAN,accessory,0,4,fail


In [298]:
levels = range(0,4)
item_levels = range(0,2)

fails = enchanting.result == 'fail'
wins = enchanting.result == 'win'

luck_2 = enchanting.luck == '2'
luck_4 = enchanting.luck == '4'

il = lambda x: enchanting.item_level == x
el = lambda x: enchanting.enchanting_level == x
i = lambda x: enchanting.item == x
l = lambda x: enchanting.luck == x

In [299]:
def ratios(item, item_level, enchanting_level, luck):
    fail = enchanting[i(item)][l(luck)][il(item_level)][el(enchanting_level)]['result'].count()
    win = enchanting[i(item)][l(luck)][il(item_level)][el(enchanting_level)]['result'].count()
    return (win+fail), win*1.0/(win+fail)

In [300]:
def ratios_ignore_luck(item, item_level, enchanting_level):
    fail = enchanting[fails][i(item)][il(item_level)][el(enchanting_level)]['result'].count()
    win = enchanting[wins][i(item)][il(item_level)][el(enchanting_level)]['result'].count()
    return (win+fail), win*1.0/(win+fail)

In [302]:
for enchanting_level in range(4,25):
    print enchanting_level, ratios_ignore_luck('weapon',14,enchanting_level)

4 (0, nan)
5 (0, nan)
6 (0, nan)
7 (0, nan)
8 (0, nan)
9 (0, nan)
10 (0, nan)
11 (0, nan)
12 (0, nan)
13 (0, nan)
14 (0, nan)
15 (1, 0.0)
16 (1, 0.0)
17 (1, 0.0)
18 (1, 0.0)
19 (1, 0.0)
20 (1, 0.0)
21 (1, 0.0)
22 (1, 0.0)
23 (1, 1.0)
24 (0, nan)


In [305]:
ratios_ignore_luck('accessory',1,2)

(19, 0.052631578947368418)